## Classification

In this lecture, we will bring together various techniques for feature engineering that we have covered in previsous sections to tackle a classification problem. This would give you an idea of the end-to-end pipeline to build machine learning algorithms for classification. 

We will:
- build a gradient boosted tree
- use feature-engine for the feature engineering steps
- set up an entire engineering and prediction pipeline using a Scikit-learn Pipeline

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# for the model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, f1_score
#from sklearn.pipeline import Pipeline

# for feature engineering
#from feature_engine import missing_data_imputers as mdi
#from feature_engine import discretisers as dsc
#from feature_engine import categorical_encoders as ce

In [22]:
# load dataset

cols = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare','embarked', 'survived']

data = pd.read_csv('../datasets/titanic.csv', usecols=cols)

data.head(10)

,pclass,survived,sex,age,sibsp,parch,fare,embarked
0,1,1,female,29.0000,0,0,211.3375,S
1,1,1,male,0.9167,1,2,151.5500,S
2,1,0,female,2.0000,1,2,151.5500,S
3,1,0,male,30.0000,1,2,151.5500,S
4,1,0,female,25.0000,1,2,151.5500,S
5,1,1,male,48.0000,0,0,26.5500,S
6,1,1,female,63.0000,1,0,77.9583,S
7,1,0,male,39.0000,0,0,0.0000,S
8,1,1,female,53.0000,2,0,51.4792,S
9,1,0,male,71.0000,0,0,49.5042,C


### Types of variables

Let's find out what types of variables there are in this dataset

In [7]:
# let's inspect the type of variables in pandas

data.dtypes

pclass        int64
survived      int64
sex          object
age         float64
sibsp         int64
parch         int64
fare        float64
embarked     object
dtype: object

There are categorical and numerical variables.

In [8]:
# let's inspect the variable values

for var in data.columns:
    print(var, data[var].unique()[0:20], '\n')

pclass [1 2 3] 

survived [1 0] 

sex ['female' 'male'] 

age [29.      0.9167  2.     30.     25.     48.     63.     39.     53.
 71.     47.     18.     24.     26.     80.         nan 50.     32.
 36.     37.    ] 

sibsp [0 1 2 3 4 5 8] 

parch [0 2 1 4 3 5 6 9] 

fare [211.3375 151.55    26.55    77.9583   0.      51.4792  49.5042 227.525
  69.3     78.85    30.      25.925  247.5208  76.2917  75.2417  52.5542
 221.7792  26.      91.0792 135.6333] 

embarked ['S' 'C' nan 'Q'] 



There continuous and discrete variables and also mixed variables.

In [27]:
# make list of variables types

# numerical: discrete vs continuous
discrete = [var for var in data.columns if data[var].dtype!='O' and var!='survived' and data[var].nunique()<10]
continuous = [var for var in data.columns if data[var].dtype!='O' and var!='survived' and var not in discrete]

# categorical
categorical = [var for var in data.columns if data[var].dtype=='O']

print('There are {} discrete variables'.format(len(discrete)))
print('There are {} continuous variables'.format(len(continuous)))
print('There are {} categorical variables'.format(len(categorical)))

There are 3 discrete variables
There are 2 continuous variables
There are 2 categorical variables


In [28]:
discrete

['pclass', 'sibsp', 'parch']

In [29]:
continuous

['age', 'fare']

In [30]:
categorical

['sex', 'embarked']

### Variable characteristics

In [31]:
# missing data

data.isnull().mean()

pclass      0.000000
survived    0.000000
sex         0.000000
age         0.200917
sibsp       0.000000
parch       0.000000
fare        0.000764
embarked    0.001528
dtype: float64

In [32]:
# cardinality (number of different categories)

data[categorical].nunique()

sex         2
embarked    3
dtype: int64

In [19]:
# feature magnitude

data.describe()

,pclass,survived,age,sibsp,parch,fare
count,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,2.294882,0.381971,29.881135,0.498854,0.385027,33.295479
std,0.837836,0.486055,14.413500,1.041658,0.865560,51.758668
min,1.000000,0.000000,0.166700,0.000000,0.000000,0.000000
25%,2.000000,0.000000,21.000000,0.000000,0.000000,7.895800
50%,3.000000,0.000000,28.000000,0.000000,0.000000,14.454200
75%,3.000000,1.000000,39.000000,1.000000,0.000000,31.275000
max,3.000000,1.000000,80.000000,8.000000,9.000000,512.329200


Features are in different ranges or scales. But this is not relevant for gradient boosted trees. 

In [23]:
# separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(
    data.drop('survived', axis=1),  # predictors
    data['survived'],  # target
    test_size=0.25,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

X_train.shape, X_test.shape

((981, 7), (328, 7))

### Missing data imputation

In [24]:
# numerical

X_train.select_dtypes(exclude='O').isnull().mean()

pclass    0.000000
age       0.193680
sibsp     0.000000
parch     0.000000
fare      0.001019
dtype: float64

In [25]:
# categorical

X_train.select_dtypes(include='O').isnull().mean()

sex         0.000000
embarked    0.002039
dtype: float64

Imputation methods I will perform:

- Numerical: arbitrary value imputation
- Categorical: add missing label imputation

### Categorical encoding and rare labels

Now that I extracted the numerical and categorical part from cabin, its cardinality is not so high.

In [22]:
# check variable frequency

var = 'cabin_cat'
(X_train[var].value_counts() / len(X_train)).sort_values()

T    0.000849
G    0.003396
A    0.016978
F    0.016978
E    0.033107
D    0.033956
B    0.049236
C    0.073854
Name: cabin_cat, dtype: float64

Categories T and G appear only in few observations, so I will replace them into rare.

### Putting it all together

In [ ]:
titanic_pipe = Pipeline([

    # missing data imputation - section 4
    ('imputer_num',
     mdi.ArbitraryNumberImputer(arbitrary_number=-1,
                                variables=['age', 'fare', 'cabin_num'])),
    ('imputer_cat',
     mdi.CategoricalVariableImputer(variables=['embarked', 'cabin_cat'])),

    # categorical encoding - section 6
    ('encoder_rare_label',
     ce.RareLabelCategoricalEncoder(tol=0.01,
                                    n_categories=6,
                                    variables=['cabin_cat'])),
    ('categorical_encoder',
     ce.OrdinalCategoricalEncoder(encoding_method='ordered',
                                  variables=['cabin_cat', 'sex', 'embarked'])),

    # Gradient Boosted machine
    ('gbm', GradientBoostingClassifier(random_state=0))
])

In [33]:
gbc = GradientBoostingClassifier(random_state=0)

In [34]:
# let's fit the pipeline and make predictions
gbc = gbc.fit(X_train, y_train)

X_train_preds = gbc.predict_proba(X_train)[:,1]
X_test_preds = gbc.predict_proba(X_test)[:,1]

ValueError: could not convert string to float: 'male'

In [26]:
# a peek into the prediction values
X_train_preds

array([0.27876558, 0.08146821, 0.46816053, ..., 0.70745459, 0.90809699,
       0.4980871 ])

In [27]:
print('Train set')
print('GBM roc-auc: {}'.format(roc_auc_score(y_train, X_train_preds)))

print('Test set')
print('GBM roc-auc: {}'.format(roc_auc_score(y_test, X_test_preds)))

Train set
GBM roc-auc: 0.9269576293912577
Test set
GBM roc-auc: 0.8396084337349397
